In [1]:
from pwn import *

def recv_options():
    for _ in range(4):
        r.recvline()

def send_opt_1(msg):
    r.sendline(b"1")
    r.recvline() # Who are you?
    r.sendline(msg.encode())
    sig_r, sig_s = r.recvline().decode()[7:-2].split(', ')
    return int(sig_r), int(sig_s)
        
r = remote('edu-ctf.csie.org', 42072)
Px, Py = r.recvline().decode()[5:-2].split(', ')

[x] Opening connection to edu-ctf.csie.org on port 42072
[x] Opening connection to edu-ctf.csie.org on port 42072: Trying 140.112.31.97
[+] Opening connection to edu-ctf.csie.org on port 42072: Done


In [2]:
Px, Py

('3694690606271234648335516341082180735031998971265994520783787436972077020228',
 '76640663065541347363104690365211313286912450815730095582626100408054127519743')

In [3]:
from ecdsa import SECP256k1
from ecdsa import ellipticcurve
from ecdsa.ecdsa import Public_key, Private_key, Signature

E = SECP256k1
G, n = E.generator, E.order
pubkey = Public_key(G, ellipticcurve.PointJacobi(E, Px, Py, 1, n))

In [4]:
recv_options()
r1, s1 = send_opt_1('123')
recv_options()
r2, s2 = send_opt_1('456')

In [5]:
(r1, s1), (r2, s2)

((106879156232169727648503995456118462580761150081527133498521700041243658767836,
  56080650897323811098342833033881064544021776789463765070122748273786405080312),
 (64608266994908543902845096126557474547352467783521818985433303673236778580963,
  37285931420404017498940420911505557984411336833474505159827745752595961430283))

In [6]:
#s1k = h1 + dr1
#s2k*1337 = h2 + dr2
#s1/(s2*1337) = (h1+dr1)/(h2+dr2)
#s1h2 + d(s1r2) = s2h1*1337 + d(s2r1*1337)

from hashlib import sha256
from Crypto.Util.number import *

h1 = bytes_to_long(sha256('123'.encode()).digest())
h2 = bytes_to_long(sha256('456'.encode()).digest())

d = (s1*h2 - (s2*h1*1337)%n) * inverse_mod((s2*r1*1337)%n - s1*r2, n) % n
d

9061930293378756934095533209556921287471686011294488665800952940561490711740

In [7]:
k = ((h1 + d * r1) % n * inverse_mod(s1, n)) % n
(k * s1) % n == (h1 + d * r1) % n

True

In [8]:
Public_key(G, d*G).point.x(), Px

(mpz(3694690606271234648335516341082180735031998971265994520783787436972077020228),
 '3694690606271234648335516341082180735031998971265994520783787436972077020228')

In [9]:
k = k * 1337 % n
k = k * 1337 % n
prikey = Private_key(pubkey, d)
sig = prikey.sign(bytes_to_long(sha256('Kuruwa'.encode()).digest()), k)
str(sig.r), str(sig.s)

('107916852895090971449850594212497868486077173959981228985777055085134873810534',
 '69532732906141280582255207042887723127073853570429222830937321134472249793400')

In [10]:
r.recvline()
r.recvline()
r.recvline()
r.recvline()

b'3) exit\n'

In [11]:
r.sendline(b"2")
r.sendline(b"Kuruwa")
r.sendline(str(sig.r).encode())
r.sendline(str(sig.s).encode())

In [12]:
r.recvline()

b'username: r: s: FLAG{fcf8ab2bc7b42bbd00e5be2b3d311ec6e8a89526}\n'